In [1]:
!pip install osmnx -q

In [2]:
from collections.abc import Callable

from typing import Tuple
from typing import Union
from typing import List
from typing import Any
from typing import Dict

from networkx import MultiDiGraph
from networkx import DiGraph

from matplotlib import pyplot

import networkx
import warnings
import random
import osmnx
import json
import time
import copy
import ast
import os

warnings.filterwarnings('ignore')

In [3]:
osmnx.config(use_cache = True, log_console = True)

ROOT_FOLDER = os.getcwd()
OUTPUT_FOLDER = os.path.join(ROOT_FOLDER, 'osm-network')
ZIP_FOLDER = os.path.join(ROOT_FOLDER, 'osm-network.zip')

RUN_TESTS = True

## 1. Downloading and loading networks with OSMNX

In [4]:
#
# Networks (generated by this code)
# https://mega.nz/file/4gBT3BLA#EYhBJmAyZFAFHsunvo_4pSFgBIuq5dGlwvQknzbYy1s
#

def download_network (output_folder : str, country : str) -> Union[MultiDiGraph, DiGraph] :
    if not os.path.exists(output_folder) :
        os.makedirs(output_folder)

    file_basic = os.path.join(output_folder, f'{country.lower()}-basic.json')
    file_exten = os.path.join(output_folder, f'{country.lower()}-extended.json')
    file_image = os.path.join(output_folder, f'{country.lower()}.png')
    file_graph = os.path.join(output_folder, f'{country.lower()}.osm')

    timer = time.perf_counter()

    MAJOR_ROAD_FILTER = (
        f'["highway"]["area"!~"yes"]'
        f'["highway"!~"residential|escape|secondary_link|tertiary_link|living_street|crossing|speed_camera|traffic_signals|trailhead|stop|bus_stop|busway|toll_gantry|abandoned|traffic_mirror|bridleway|street_lamp|bus_guideway|construction|corridor|cycleway|elevator|escalator|footway|milestone|path|pedestrian|planned|platform|proposed|raceway|service|steps|track|emergency_bay|give_way"]'
        f'["service"!~"alley|driveway|emergency_access|parking|parking_aisle|private"]'
    )

    g = osmnx.graph_from_place(country, custom_filter = MAJOR_ROAD_FILTER)

    osmnx.plot_graph(g, filepath = file_image, show = False, save = True, dpi = 300)

    g = osmnx.distance.add_edge_lengths(g)
    g = osmnx.add_edge_speeds(g)
    g = osmnx.add_edge_travel_times(g)

    with open(file_basic, 'w') as file :
        json.dump(osmnx.basic_stats(g), file, indent = 4)

    with open(file_exten, 'w') as file :
        json.dump(osmnx.extended_stats(g), file, indent = 4)

    osmnx.save_graphml(g, file_graph)

    print(country)
    print(f'Nodes : {len(list(g.nodes))}')
    print(f'Edges : {len(list(g.edges))}')
    print(f'Timer : {(time.perf_counter() - timer):.2f} seconds')
    print()

    return g

def load_network (filepath : str, single_graph : bool = True) -> Union[MultiDiGraph, DiGraph] :
    g = networkx.read_graphml(filepath)

    dtypes = {
        "elevation": float,
        "elevation_res": float,
        "lat": float,
        "lon": float,
        "osmid": int,
        "street_count": int,
        "x": float,
        "y": float,
    }

    for _, data in g.nodes(data = True):
            for attr in data.keys() & dtypes.keys():
                data[attr] = dtypes[attr](data[attr])

    dtypes = {
        "bearing": float,
        "grade": float,
        "grade_abs": float,
        "length": float,
        "osmid": int,
        "speed_kph": float,
        "travel_time": float,
    }

    for _, _, data in g.edges(data = True):
        data.pop("id", None)

        for attr, value in data.items():
            if value.startswith("[") and value.endswith("]"):
                try:
                    data[attr] = ast.literal_eval(value)
                except (SyntaxError, ValueError):
                    pass

        for attr in data.keys() & dtypes.keys():
            if isinstance(data[attr], list):
                data[attr] = [dtypes[attr](item) for item in data[attr]]
            else:
                data[attr] = dtypes[attr](data[attr])

    if single_graph :
        return DiGraph(g)

    return g

In [5]:
graphs = dict()

if not os.path.exists(ZIP_FOLDER) :
    for country in ['Luxembourg', 'Montenegro', 'Slovenia', 'Belgium', 'Netherlands'] :
        print(f'Downloading network for {country} ...')

        graphs[country.lower()] = download_network(output_folder = OUTPUT_FOLDER, country = country)

    print()
    print(f'Ziping dowloaded networks to : {ZIP_FOLDER}')

    !zip -q -r osm-network.zip osm-network/
else :
    if not os.path.exists(OUTPUT_FOLDER) :
        print(f'Unziping downloaded networks to : {OUTPUT_FOLDER}')
        print()

        !unzip -q osm-network.zip

    for country in ['Luxembourg', 'Montenegro', 'Slovenia', 'Belgium', 'Netherlands'] :
        print(f'Loading network for {country} ...')

        path = os.path.join(OUTPUT_FOLDER, f'{country.lower()}.osm')

        graphs[country.lower()] = load_network(filepath = path, single_graph = True)

Loading network for Luxembourg ...
Loading network for Montenegro ...
Loading network for Slovenia ...
Loading network for Belgium ...
Loading network for Netherlands ...


## 2. Checking network attributes

In [6]:
graph = graphs['netherlands']

print(graph)
print()

print('Node attributes : ')
for _, data in graph.nodes.data() :
    print(json.dumps(data, indent = 4))
    break

print()

print('Edge attributes : ')
for _, _, data in graph.edges.data() :
    print(json.dumps(data, indent = 4))
    break

DiGraph with 252752 nodes and 559879 edges

Node attributes : 
{
    "y": 51.24191,
    "x": 3.45908,
    "street_count": 1
}

Edge attributes : 
{
    "osmid": [
        7629772,
        290141909
    ],
    "name": "Graaf Jansdijk",
    "highway": "unclassified",
    "oneway": "False",
    "length": 93.033,
    "geometry": "LINESTRING (3.45908 51.24191, 3.4603701 51.2419384, 3.4604106 51.2419882)",
    "speed_kph": 48.4,
    "travel_time": 6.9
}


## 3. Random node attacks on directed network

In [7]:
def remove_random_nodes (graph : Union[MultiDiGraph, DiGraph], percentage : float = 0.01) -> Union[MultiDiGraph, DiGraph] :
    graph = copy.deepcopy(graph)

    threshold = round(percentage * graph.number_of_nodes())
    target = random.sample(list(graph.nodes), threshold)

    print(f'Removing {threshold} randomly selected nodes.')

    graph.remove_nodes_from(target)

    return graph

In [8]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        for i in [0.50] :
            timer = time.perf_counter()

            g = remove_random_nodes(graph = graph, percentage = i)

            print(g)
            print(f'{country[0].upper()}{country[1:]:<10s} - {i:4.2f} ~~ {(time.perf_counter() - timer):5.2f} seconds')
            print()

DiGraph with 4030 nodes and 9181 edges

Removing 2015 randomly selected nodes.
DiGraph with 2015 nodes and 2291 edges
Montenegro  - 0.50 ~~  0.23 seconds

DiGraph with 252752 nodes and 559879 edges

Removing 126376 randomly selected nodes.
DiGraph with 126376 nodes and 140664 edges
Netherlands - 0.50 ~~ 16.15 seconds



## 4. Random edge attacks on directed networks

In [9]:
def remove_random_edges (graph : Union[MultiDiGraph, DiGraph], percentage : float = 0.01) -> Union[MultiDiGraph, DiGraph] :
    graph = copy.deepcopy(graph)

    threshold = round(percentage * graph.number_of_edges())
    target = random.sample(list(graph.edges), threshold)

    print(f'Removing {threshold} randomly selected edges.')

    graph.remove_edges_from(target)

    return graph

In [10]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        for i in [0.50] :
            timer = time.perf_counter()

            g = remove_random_edges(graph = graph, percentage = i)

            print(g)
            print(f'{country[0].upper()}{country[1:]:<10s} - {i:4.2f} ~~ {(time.perf_counter() - timer):5.2f} seconds')
            print()

DiGraph with 4030 nodes and 9181 edges

Removing 4590 randomly selected edges.
DiGraph with 4030 nodes and 4591 edges
Montenegro  - 0.50 ~~  0.22 seconds

DiGraph with 252752 nodes and 559879 edges

Removing 279940 randomly selected edges.
DiGraph with 252752 nodes and 279939 edges
Netherlands - 0.50 ~~ 19.25 seconds



## 5. Random walk attacks on directed networks

In [11]:
def remove_random_walks (graph : Union[MultiDiGraph, DiGraph], percentage : float = 0.05, max_walk_length : int = 5) -> Tuple[Union[MultiDiGraph, DiGraph], float] :
    graph = copy.deepcopy(graph)

    threshold = round(percentage * graph.number_of_edges())
    target = set()
    active = None

    sum_length = 0
    n_length = 0

    length = 0
    repeat = 0

    while len(target) < threshold :
        if repeat > 3 :
            active = None

        if active is None :
            active = random.choice(list(graph.nodes))

        edges = list(graph.out_edges(active))

        if len(edges) > 0 :
            edge = random.choice(edges)

            if edge not in target :
                target.add(edge)

                length = length + 1
                repeat = 0

                active = edge[1]
            else :
                repeat = repeat + 1

            if length >= max_walk_length :
                sum_length = sum_length + length
                n_length = n_length + 1

                length = 0
                repeat = 0
                active = None
        else :
            sum_length = sum_length + length
            n_length = n_length + 1

            length = 0
            repeat = 0
            active = None

    print(f'Removing {threshold} semi-randomly selected edges with max walk length of {max_walk_length}.')

    graph.remove_edges_from(list(target))

    return graph, sum_length / n_length if n_length > 0 else 0

In [12]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        for l in [25] :
            for i in [0.50] :
                timer = time.perf_counter()

                g, avg_l = remove_random_walks(graph = graph, percentage = i, max_walk_length = l)

                print(g)
                print(f'{country[0].upper()}{country[1:]:<10s} - {i:4.2f} {avg_l:5.2f} ~~ {(time.perf_counter() - timer):7.2f} seconds')
                print()

DiGraph with 4030 nodes and 9181 edges

Removing 4590 semi-randomly selected edges with max walk length of 25.
DiGraph with 4030 nodes and 4591 edges
Montenegro  - 0.50 24.34 ~~    0.32 seconds

DiGraph with 252752 nodes and 559879 edges

Removing 279940 semi-randomly selected edges with max walk length of 25.
DiGraph with 252752 nodes and 279939 edges
Netherlands - 0.50 24.26 ~~  701.31 seconds



## 6. Directed edge attack on directed networks

In [13]:
def increment_dict (table : Dict[str, int], key : str) -> None :
    if key in table.keys() :
        table[key] = table[key] + 1
    else :
        table[key] = 1

def edge_type_distribution (graph : Union[MultiDiGraph, DiGraph]) -> None :
    stats = dict()

    for x, y, data in graph.edges.data() :
        if 'highway' in data.keys() :
            highway = data['highway']

            if type(highway) == str :
                increment_dict(table = stats, key = highway)
            elif type(highway) == list :
                for name in highway :
                    increment_dict(table = stats, key = name)
            else :
                increment_dict(table = stats, key = 'error')
        else :
            increment_dict(table = stats, key = 'n/a')

    print(json.dumps(stats, indent = 4))

In [14]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        edge_type_distribution(graph = graph)

        print()

DiGraph with 4030 nodes and 9181 edges

{
    "unclassified": 5220,
    "secondary": 1022,
    "tertiary": 1189,
    "primary": 1652,
    "primary_link": 112,
    "road": 6
}

DiGraph with 252752 nodes and 559879 edges

{
    "unclassified": 334027,
    "tertiary": 113024,
    "primary": 36840,
    "secondary": 61491,
    "trunk": 3301,
    "primary_link": 1838,
    "trunk_link": 1580,
    "motorway_link": 4880,
    "motorway": 3449,
    "road": 44,
    "rest_area": 63,
    "passing_place": 4
}



In [15]:
def remove_direct_edges (graph : Union[MultiDiGraph, DiGraph], edges : List[Any]) -> Union[MultiDiGraph, DiGraph] :
    graph = copy.deepcopy(graph)

    print(f'Removing {len(edges)} directly selected edges.')

    graph.remove_edges_from(edges)

    return graph

def filter_edges (graph : Union[MultiDiGraph, DiGraph], filter : str = 'motorway') -> List[Tuple[Any, Any]] :
    edges = list()

    for x, y, data in graph.edges.data() :
        if 'highway' in data.keys() :
            highway = data['highway']

            if type(highway) == str :
                if highway.replace('"', '').startswith(filter) :
                    edges.append((x, y))
            elif type(highway) == list :
                for name in highway :
                    if name.replace('"', '').startswith(filter) :
                        edges.append((x, y))

    return edges

In [16]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        for filter in ['motorway', 'trunk', 'primary'] :
            timer = time.perf_counter()

            edges = filter_edges(graph = graph, filter = filter)

            i = 100.0 * len(edges) / graph.number_of_edges()

            g = remove_direct_edges(graph = graph, edges = edges)

            print(g)
            print(f'{country[0].upper()}{country[1:]:<10s} - {filter:8s} {i:5.2f} % ~~ {(time.perf_counter() - timer):7.2f} seconds')
            print()

DiGraph with 4030 nodes and 9181 edges

Removing 0 directly selected edges.
DiGraph with 4030 nodes and 9181 edges
Montenegro  - motorway  0.00 % ~~    0.21 seconds

Removing 0 directly selected edges.
DiGraph with 4030 nodes and 9181 edges
Montenegro  - trunk     0.00 % ~~    0.48 seconds

Removing 1764 directly selected edges.
DiGraph with 4030 nodes and 7417 edges
Montenegro  - primary  19.21 % ~~    0.23 seconds

DiGraph with 252752 nodes and 559879 edges

Removing 8329 directly selected edges.
DiGraph with 252752 nodes and 551551 edges
Netherlands - motorway  1.49 % ~~   18.49 seconds

Removing 4881 directly selected edges.
DiGraph with 252752 nodes and 555002 edges
Netherlands - trunk     0.87 % ~~   17.77 seconds

Removing 38678 directly selected edges.
DiGraph with 252752 nodes and 521207 edges
Netherlands - primary   6.91 % ~~   17.74 seconds



## 7. Directed node attack on directed networks

In [17]:
def increment_dict (table : Dict[str, int], key : str) -> None :
    if key in table.keys() :
        table[key] = table[key] + 1
    else :
        table[key] = 1

def node_type_distribution (graph : Union[MultiDiGraph, DiGraph]) -> None :
    stats = dict()

    for x, data in graph.nodes.data() :
        if 'street_count' in data.keys() :
            degree = data['street_count']

            if type(degree) == str :
                increment_dict(table = stats, key = degree)
            elif type(degree) == int :
                increment_dict(table = stats, key = degree)
            else :
                increment_dict(table = stats, key = 'error')
        else :
            increment_dict(table = stats, key = 'n/a')

    print(json.dumps(stats, indent = 4))

In [18]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        node_type_distribution(graph)

        print()

DiGraph with 4030 nodes and 9181 edges

{
    "3": 2898,
    "1": 922,
    "4": 170,
    "2": 40
}

DiGraph with 252752 nodes and 559879 edges

{
    "1": 48729,
    "3": 167943,
    "4": 28900,
    "2": 6629,
    "5": 503,
    "6": 44,
    "8": 2,
    "7": 2
}



In [19]:
def remove_direct_nodes (graph : Union[MultiDiGraph, DiGraph], nodes : List[Any]) -> Union[MultiDiGraph, DiGraph] :
    graph = copy.deepcopy(graph)

    print(f'Removing {len(nodes)} directly selected nodes.')

    graph.remove_nodes_from(nodes)

    return graph

def filter_nodes (graph : Union[MultiDiGraph, DiGraph], min_degree : int = 5) -> List[Any] :
    nodes = list()

    for x, data in graph.nodes.data() :
        if 'street_count' in data.keys() and data['street_count'] >= min_degree :
            nodes.append(x)

    return nodes

In [20]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        for k in [4] :
            timer = time.perf_counter()

            nodes = filter_nodes(graph = graph, min_degree = k)

            i = 100.0 * len(nodes) / graph.number_of_nodes()

            g = remove_direct_nodes(graph = graph, nodes = nodes)

            print(g)
            print(f'{country[0].upper()}{country[1:]:<10s} - {k} {i:5.2f} % ~~ {(time.perf_counter() - timer):7.2f} seconds')
            print()

DiGraph with 4030 nodes and 9181 edges

Removing 170 directly selected nodes.
DiGraph with 3860 nodes and 8258 edges
Montenegro  - 4  4.22 % ~~    0.20 seconds

DiGraph with 252752 nodes and 559879 edges

Removing 29451 directly selected nodes.
DiGraph with 223301 nodes and 414766 edges
Netherlands - 4 11.65 % ~~   17.70 seconds



## 8. Directed node attacks on directed sub-networks

In [21]:
def create_subgraph (graph : Union[MultiDiGraph, DiGraph], percentage : float = 0.1) -> Union[MultiDiGraph, DiGraph] :
    threshold = round(percentage * graph.number_of_nodes())

    processed = set()
    pending = list()
    nodes = list()

    node = random.choice(list(graph.nodes))

    pending.append(node)
    processed.add(node)
    nodes.append(node)

    while len(nodes) < threshold :
        if len(pending) < 1 :
            break

        node = pending[0]
        pending = pending[1:]

        edges = list(graph.edges(node))

        for source, target in edges :
            if target not in processed :
                pending.append(target)
                processed.add(target)
                nodes.append(target)

    graph = graph.subgraph(nodes).copy()

    return graph

def find_node_importance (graph : Union[MultiDiGraph, DiGraph], method : str = 'betwenness') -> Any :
    if method == 'betwenness' :
        return networkx.betweenness_centrality(graph)

    raise ValueError()

def remove_direct_nodes_subgraph (graph : Union[MultiDiGraph, DiGraph], percentage : float = 0.1, subgraph_percentage : float = 0.01, subgraph_count : int = 100) -> Union[MultiDiGraph, DiGraph] :
    threshold = round(percentage * graph.number_of_nodes())

    nodes = dict()

    for i in range(subgraph_count) :
        g = create_subgraph(graph = graph, percentage = subgraph_percentage)
        n = find_node_importance(graph = g, method = 'betwenness')

        for key, value in n.items() :
            if key in nodes.keys() :
                if value > nodes[key] :
                    nodes[key] = value
            else :
                nodes[key] = value

    nodes = [(key, value) for key, value in nodes.items()]
    nodes = sorted(nodes, key = lambda x : x[1], reverse = True)
    nodes = nodes[:threshold]
    nodes = [k for k, _ in nodes]

    return remove_direct_nodes(graph = graph, nodes = nodes)

In [22]:
if RUN_TESTS :
    for country in ['montenegro', 'netherlands'] :
        graph = graphs[country]

        print(graph)
        print()

        for i in [0.50] :
            timer = time.perf_counter()

            g = remove_direct_nodes_subgraph(
                graph = graph,
                percentage = i, 
                subgraph_percentage  = 0.01,
                subgraph_count = 100
            )

            print(g)
            print(f'{country[0].upper()}{country[1:]:<10s} - {i:4.2f} % ~~ {(time.perf_counter() - timer):7.2f} seconds')
            print()

DiGraph with 4030 nodes and 9181 edges

Removing 2015 directly selected nodes.
DiGraph with 2015 nodes and 3580 edges
Montenegro  - 0.50 % ~~    0.83 seconds

DiGraph with 252752 nodes and 559879 edges

Removing 126376 directly selected nodes.
DiGraph with 126376 nodes and 226608 edges
Netherlands - 0.50 % ~~ 3119.49 seconds



## 11. Ploting

In [23]:
def plot (data : Any, xlabel : str = '', ylabel : str = '', title : str = '', save : bool = True, filename : str = 'plot.png') -> None :
    # TODO

    pyplot.xlabel(xlabel)
    pyplot.ylabel(ylabel)

    if save :
        pyplot.savefig(filename, dpi = 300, format = 'png')

    pyplot.show()